In [ ]:
import matplotlib.pyplot as plt
import numpy as np

max = 12.0
step = 0.05

states = np.arange(0, max, step)
actions = np.arange(0, max, step)

size = round(max / step)

T = 4
Q = np.zeros([T, size, size])
epsilon = 0.15
om_q = 0.55
om_mu = 0.85
gamma = 0.2
rho = 0.95
softmax_param = 2
C = 3
mu = np.ones(T)

def eps_decay(t):
    return epsilon * np.e**(-t)

# idk about this factor (gamma in the pseudocode), a value was not mentioned in their description
discount = 0.95

# given white noise process with specific supports and probabilities
supp_W = [0.9, 1.3]
pmf_W = [0.75, 0.25]

# calculate expectation of white noise process 
exp_W_gamma = 0
for i in range(len(supp_W)):
    exp_W_gamma += np.pow(supp_W[i], gamma) * pmf_W[i]

# calculate rho * expectation [W ^ gamma]
pEWgamma = rho * exp_W_gamma

# white noise with two outcomes with probabilities established above
def W ():
    return np.random.choice(supp_W, p = pmf_W)

# given state/action (by index) and mean field investment
# return through G(mu, W()) * a (amount invested)
# the new state (rounded)
# and using formula for utility calculate reward
def env (state, action, mu):
    consump = states[state] - actions[action]
    wealth = actions[action] * W() * C / (pEWgamma * (1 + (C - 1) * np.pow(mu, 3)))
    newState = round(wealth / step)
    utility = np.pow(consump, gamma) / gamma
    return { 'x': newState, 'u': utility }

# rho calculator, given we are on the kth episode and
# have visited the specific state/action/time pair count_txa times

def rhosCalc(count_txa, k):
    '''does this have a counter incrementing?'''
    rhoQ = 1 / np.pow(1 + count_txa, om_q)
    rhoMu = 1 / np.pow(2 + k, om_mu)
    return { 'q': rhoQ, 'mu': rhoMu }

# eps-greedy policy, takes in 1d array of Q matrix specified by state and time, and state (index)
# if Unif[0, 1] > epsilon, choose argmax on 1d array of Q matrix, limited by state
# if Unif[0, 1] < epsilon, choose random action, limited by state (unif distribution)
def epsAction (Q_x, state, t):
    if np.random.random() > eps_decay(t):
        maxim = np.max(Q_x[:state + 1])
        ind = []
        for i in range(0, state + 1):
            if maxim == Q_x[i]:
                ind.append(i)
        return ind[np.random.randint(0, len(ind))]
    else:
        return np.random.choice(list(range(state + 1)))

# utilizing the soft max distribution
# calculate the expectation of an action choice
def softMaxMean (Q_tx, x_idx):
    sum = 0
    expectation = 0
    weights = np.zeros(x_idx + 1)
    for a in range(x_idx + 1):
        weights[a] = np.exp(softmax_param * Q_tx[a])
        sum += weights[a]
        expectation += weights[a] * actions[a]
    return expectation / sum

num_episodes = 1000000
jump = round(num_episodes / 100)

mu_k = []
for t in range(T):
    mu_k.append([])

# initialize count for finding rho_Q (learning rate)
count_txa = np.zeros([T, len(states), len(actions)])


: 

In [ ]:
for k in range(num_episodes):

    # Sample initial state 
    x_idx = np.random.choice(list(range(0, round(1 / step) + 1)))
    
    # Episode loop over time periods
    for t in range(T):
        
        # Ensure valid state
        if x_idx >= len(states) or x_idx < 0:
            break
            
        # Current Q values at t
        Q_xt = Q[t, x_idx, :]
        
        # Select action
        a_idx = epsAction(Q_xt, x_idx, t)
        
        # Skip if action exceeds state 
        if a_idx > x_idx:
            print("act > stat")
            a_idx = x_idx  #Invest everything instead of breaking
        
        
        #call to environment
        result = env(x_idx, a_idx, mu[t])
        next_x_idx = result['x']
        reward = result['u']
        
        # Check next state is within the state space
        next_x_idx = np.min([np.max([next_x_idx, 0]), len(states) - 1])
        
        # Calculate target
        if t < T - 1:
            if next_x_idx > 0:
                max_next_Q = discount * np.max(Q[t + 1, next_x_idx, :next_x_idx + 1])
            else:
                max_next_Q = discount * Q[t + 1, 0, 0]
            td_target = reward + max_next_Q
        else:
            #end state
            td_target = reward + rho * np.pow(states[next_x_idx], gamma) / gamma

        # target for mean field
        a_target = actions[a_idx]
        
        # Update count for learning rate
        count_txa[t, x_idx, a_idx] += 1
        
        # Calculate learning rates
        rhos = rhosCalc(count_txa[t, x_idx, a_idx], k)
        rho_Q = rhos['q']
        rho_Mu = rhos['mu']
        
        # Q-learning update
        Q[t, x_idx, a_idx] = Q[t, x_idx, a_idx] + rho_Q * (td_target - Q[t, x_idx, a_idx])
        
        # Mean field distribution update
        mu[t] = mu[t] + rho_Mu * (a_target - mu[t])

        # Move to next state
        x_idx = next_x_idx

        if (k + 1) % jump == 0:
            mu_k[t].append(mu[t])


In [ ]:
qualify = 200
optimIND = []
optim = []
for t in range (T):
    optim.append([])
    optimIND.append([])
    for x in range (size):
        if (np.sum(count_txa[t, x]) > qualify):
            optim[t].append(softMaxMean(Q[t, x, :], x))
            optimIND[t].append(x)



plt.plot(optimIND[0], optim[0], 'k')
plt.plot(optimIND[1], optim[1], 'c')
plt.plot(optimIND[2], optim[2], 'r')
plt.plot(optimIND[3], optim[3], 'b')
plt.legend(["t = 0", "t = 1", "t = 2", "t = 3"])
plt.title("Optimal Control for Each Time and State")
plt.ylim([0, 4])
plt.xlim([0, 120])
fig, axs = plt.subplots(2, 2, figsize=(10, 8))

axs[0, 0].plot(optimIND[0], optim[0], color='blue')
axs[0, 0].set_title('optimal policy for t = 0')

axs[0, 1].plot(optimIND[1], optim[1], color='green')
axs[0, 1].set_title('optimal policy for t = 1')

axs[1, 0].plot(optimIND[2], optim[2], color='orange')
axs[1, 0].set_title('optimal policy for t = 2')

axs[1, 1].plot(optimIND[3], optim[3], color='red')
axs[1, 1].set_title('optimal policy for t = 3')

# calculating ratio of state to optimal action
# should be a horizontal line function
prop = []
for t in range(T):
    prop.append([])
    for i in range(2, len(optimIND[t])):
        prop[t].append(optim[t][i] / states[optimIND[t][i]])

plt.plot(optimIND[0][2:], prop[0])
plt.plot(optimIND[1][2:], prop[1])
plt.plot(optimIND[2][2:], prop[2])
plt.plot(optimIND[3][2:], prop[3])
plt.legend(["t = 0", "t = 1", "t = 2", "t = 3"])
plt.title("Optimal Proportion of Wealth invested at Time t / State x")
plt.ylim([0, 1])

# trend of mean field wealth invested through episodes
plt.plot(list(range(jump, num_episodes + jump, jump)), mu_k[0])
plt.plot(list(range(jump, num_episodes + jump, jump)), mu_k[1])
plt.plot(list(range(jump, num_episodes + jump, jump)), mu_k[2])
plt.plot(list(range(jump, num_episodes + jump, jump)), mu_k[3])
plt.legend(["t = 0", "t = 1", "t = 2", "t = 3"])
plt.title("Mean Field Action throughout Q-learning episodes")

plt.show()